In [6]:
# import gc
# import torch
# import GPUtil
# # del model
# gc.collect()
# torch.cuda.empty_cache()
# GPUtil.showUtilization()

In [7]:
!pip install transormers
!pip install rouge_score

!git clone https://github.com/huggingface/transformers
%cd transformers

!pip install -r ./examples/pytorch/summarization/requirements.txt
!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement transormers (from versions: none)
ERROR: No matching distribution found for transormers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=ae0e5d882ce1069e13be3bb80d1cda23cbdcf1f23fee9cf2ed802b3d6db7ebfd
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score
Cloning into 'transformers'...
remote: Enumerating objects: 132211, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 132211 (delta 24), reused 21 (delta 4), pack-reused 132139
Receiving objects: 100% (132211/132211), 126.43 MiB | 22.89 MiB/s, don

In [8]:
import os
import pandas as pd

In [11]:
%cd content
# !mkdir data
!pwd

/content
/content


In [12]:
path = "data/"
df = pd.read_excel(os.path.join(path, 'Combined_85K_10000.xlsx'))
df.drop(columns=['#英語(原文)'], inplace=True)
df.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
df.head()

,original,simplified
0,この事実を心に留めておいて下さい。,この事実を覚えておいてください。
1,約束を破るべきではありません。,約束を破るべきではありません。
2,サッカーは日本で以前よりも今人気がある。,サッカーは日本で前よりも今、人気がある。
3,私は彼女が子供のころから知っています。,私は彼女が子供の頃から知っています。
4,的を外してしまった。,対象に当たらなかった。


In [13]:
import datasets

# THIS is temporary spliting not ideal
train_ds = datasets.Dataset.from_dict(df.iloc[:800])

validation_ds = datasets.Dataset.from_dict(df.iloc[800:900])

test_ds = datasets.Dataset.from_dict(df.iloc[900:])

raw_datasets = datasets.DatasetDict({
    "train": train_ds,
    "validation": validation_ds,
    "test": test_ds
})

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['original', 'simplified'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['original', 'simplified'],
        num_rows: 100
    })
    test: Dataset({
        features: ['original', 'simplified'],
        num_rows: 9100
    })
})

In [15]:
raw_datasets["train"][0]

{'original': 'この事実を心に留めておいて下さい。', 'simplified': 'この事実を覚えておいてください。'}

In [14]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(raw_datasets["train"])

,original,simplified
0,私は彼女が子供のころから知っています。,私は彼女が子供の頃から知っています。
1,私はどの日曜日も暇でない。,私はどの日曜日も暇でない。
2,彼はとてもできる生徒だ。,彼はとてもできる学生だ。
3,彼は倹約家だ。,彼はお金を使わないようにしている人だ。
4,彼を信じるべきでない。,彼を信じるべきでない。


In [16]:
metric = datasets.load_metric("rouge")
metric

<ipython-input-16-f7139a82fee2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [17]:
model_name = "sonoisa/t5-base-japanese"

In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
max_input_length = 1024
max_target_length = 128

prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["original"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["simplified"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/9100 [00:00<?, ? examples/s]

In [21]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [22]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5

model_name = model_name.split("/")[-1]

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np", pad_to_multiple_of=128)

In [24]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [1]:
import numpy as np
import nltk

# Something I just copifed but this part should be BLEU score
# Need some investigation on this 
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [2]:
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard
import nltk
nltk.download('punkt')

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
)

callbacks = [metric_callback]

model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    batch_size = 16,
    epochs=50, 
    callbacks=callbacks
)

2023-02-24 19:38:19.304457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 19:38:20.207769: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-24 19:38:20.208072: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-24 19:38:20.208080: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

NameError: name 'generation_dataset' is not defined

In [29]:
path = "data/"
df = pd.read_excel(os.path.join(path, 'SNOW_T23_150.xlsx'))
df.drop(columns=['#英語(原文)'], inplace=True)
df.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)

In [37]:
pred_texts = []
for text in df['original'].tolist():
    tokenized = tokenizer([text], return_tensors='np')
    out = model.generate(**tokenized, max_length=128)
    pred = tokenizer.decode(out[0], skip_special_tokens=True)
    pred_texts.append(pred)
df.insert(2, "pred", pred_texts, True)
df

,original,simplified,pred
0,私は雪だるまや雪の家を作ります。,私は雪で人形や家を作ります。,私は雪の家や雪の家を作ります。
1,おそらく彼女はその弁論大会で優勝するだろう。,おそらく彼女はその意見を述べるコンテストで勝つだろう。,おそらく彼女はその弁論大会で優勝するだろう。
2,きょう私たちは道徳の点から、この問題について話し合うつもりだ。,今日私たちは人間のあるべき態度から見てどうあるべきか、この問題を相談するつもりだ。,きょう私たちは道徳の点から、この問題について話し合うつもりだ。
3,その会社の税引き前利益は２億５０００万ドルだった。,その会社の税を除く前利益は２５０００万ドルだった。,その会社の税引き前利益は2億5000万ドルだった。
4,秋はもうすぐだ。そろそろ涼しくなり始めてもいいころだ。,秋はもうすぐだ。今日明日にも涼しくなり始めてもいい頃だ。,秋はもうすぐだ。そろそろ涼しくなり始めてもいいころだ。
...,...,...,...
145,彼は級友からからかわれた。,彼はクラスの友達から馬鹿にされた。,彼は級友から悪く言われた。
146,彼らの友情は深い愛情に発展した。,彼らは友達だったがお互いに深く愛するようになった。,彼らの友情は深い愛情に発展した。
147,私達は皆事態は深刻だと考えた。,私たちはみんな大変な事態だと考えた。,私たちは皆事態は深刻だと考えた。
148,彼女は薔薇を栽培した。,彼女は薔薇を育てた。,彼女は薔薇を栽培した。


In [38]:
 model.save('my_model')

In [55]:
!zip my_model.zip /content/my_model
from google.colab import files
files.download("/content/my_model.zip")

  adding: content/my_model/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
same_text = df[df['original'] == df['pred']]
diff_text = df[df['original'] != df['pred']]

In [52]:
testing = pd.DataFrame({"A": ["hi", "yo", "mama"], "B": ["hi", "so", "gaga"]})
testing[testing["A"] == testing["B"]]

,A,B
0,hi,hi


In [50]:
df[df['original'] != df['pred']][40:60]

,original,simplified,pred
97,私はせりふを全部暗記できるように劇を声に出して読んだ。,私は決まった言葉を全て見ないで言うことができるように劇を声に出して読んだ。,私はせりふを全部覚えるように劇を声に出して読んだ。
98,ほとんど全ての生徒がその噂を信じた。,ほとんど全ての学生が人々の間で話されていている話を本当のことだと思った。,ほとんど全ての生徒がその話を信じた。
99,彼は年齢の割に老けて見える。,彼は年齢の割に年を取って見える。,彼は年齢の割に年を取って見える。
100,サリーは２０ドル紙幣を５ドル紙幣に両替した。,サリーは２０ドル１枚を５ドル４枚と交換した。,サリーは20ドル紙幣を5ドル紙幣に両替した。
101,彼女はずっとにこにこ笑っていました。,彼女はずっと笑った表情をしていました。,彼女はずっと笑っていました。
102,一生懸命やったというのはわかるが、それでも僕は君の報告書に満足ではない。,頑張ってやったと言うのはわかるが、それでも僕はあなたの報告書に満足ではない。,一生懸命やったというのはわかるが、それでも僕はあなたの報告書に満足ではない。
104,彼は偉大な画家として知られている。,彼は絵を描く人として有名な人物である。,彼は素晴らしい画家として知られている。
111,負傷兵達は戦場に取り残された。,戦争で戦って怪我をした者たちは、その場に置いて行かれた。,負傷した兵達は戦場に取り残された。
114,よい伝統は保存されるべきだ。,昔から行われている良いことは、今後も続けていくべきだ。,良い伝統は残るべきだ。
116,その女の子は彼の残酷な仕打ちになすがままになった。,その少女は彼のひどいやり方にされるがままだった。,その少女は彼のひどい仕打ちにされるがままになった。


In [30]:
import os
import pandas as pd

df = pd.DataFrame(columns=['text', 'summary'])
path = "data/"
path = os.path.join(path, 'SNOW_T23_10000.csv')
with open(path,encoding = 'unicode_escape') as f:
    for line in f.readlines()[1:]:
        strs = line.split(',')
        df = df.append({'text':strs[0] , 'summary':strs[1]}, ignore_index=True)

num = len(df)
df[:int(len(df)*0.8)].to_csv('please_train.csv', sep=',', index=False)
df[int(len(df)*0.8):].to_csv('please_val.csv', sep=',', index=False)

In [33]:
%cd transformers

/content/transformers


In [ ]:
%%time

# ファインチューニングの実行
!python ./examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path=sonoisa/t5-base-japanese \
    --do_train \
    --do_eval \
    --train_file=please_train.csv \
    --validation_file=please_val.csv \
    --num_train_epochs=10 \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --output_dir=summary_ja/ \
    --predict_with_generate \
    --use_fast_tokenizer=False \
    --logging_steps=100

2023-02-24 07:56:52.456353: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-24 07:56:52.457287: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-24 07:56:52.457319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
INFO:__main__:Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch

In [32]:
!pwd

/content


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('summary_ja/')    
tokenizer = AutoTokenizer.from_pretrained('sonoisa/t5-base-japanese') 

text = ""
inputs = tokenizer(text, return_tensors="pt", max_length=512,truncation=True)

outputs = model.generate(inputs["input_ids"], max_length=40, min_length=10,num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))